In [1]:
import numpy as np
import pandas as pd

In [2]:
from DCC_GARCH.GARCH.GARCH import GARCH
from DCC_GARCH.GARCH.GARCH_loss import garch_loss_gen
from DCC_GARCH.DCC.DCC import DCC
from DCC_GARCH.DCC.DCC_loss import dcc_loss_gen

In [27]:
datadf = pd.read_csv('Compiled_Data.csv').set_index('Date').dropna()
datadf.tail()

,USTC,FRAX,AMPL,DAI,EOSDT,USDC,USDT,PAXG,XAUT
Date,,,,,,,,,
26-09-2022,0.033281,0.993318,1.170409,0.999868,0.895347,1.000165,0.999990,1619.209106,1626.883545
27-09-2022,0.031270,0.992912,1.095596,0.999813,0.882125,1.000123,0.999980,1621.632813,1629.180786
28-09-2022,0.031109,0.993726,1.155449,0.999388,0.893674,1.000011,0.999976,1648.627930,1654.329590
29-09-2022,0.030579,0.993518,1.145941,1.000085,0.910731,1.000124,1.000079,1656.146729,1660.748535
30-09-2022,0.031965,0.991634,1.165521,0.999818,0.888880,1.000066,1.000049,1655.731689,1660.823853


In [16]:
def run_dcc_model(srs1,srs2):
    Series1 = np.log(srs1).dropna().iloc[::-1] # the latest data should come first
    Series2 = np.log(srs2).dropna().iloc[::-1] # the latest data should come first

    Series1_model = GARCH(1,1)
    Series1_model.set_loss(garch_loss_gen(1, 1))
    Series1_model.set_max_itr(1)
    Series1_model.fit(Series1)
    print("Series1 Theta : ", Series1_model.get_theta())
    
    Series2_model = GARCH(1,1)
    Series2_model.set_loss(garch_loss_gen(1, 1))
    Series2_model.set_max_itr(1)
    Series2_model.fit(Series2)
    print("Series2 Theta : ", Series2_model.get_theta())
    
    Series1_sigma = Series1_model.sigma(Series1)
    Series1_epsilon = Series1 / Series1_sigma

    Series2_sigma = Series2_model.sigma(Series2)
    Series2_epsilon = Series2 / Series2_sigma

    epsilon = np.array([Series1_epsilon,Series2_epsilon])
    
    dcc_model = DCC()
    dcc_model.set_loss(dcc_loss_gen())
    dcc_model.fit(epsilon)
    return dcc_model.get_ab()

In [18]:
run_dcc_model(datadf["USTC"],datadf["PAXG"])

Iteration: 1. Training loss: 1.818E+04.
Series1 Theta :  [0.         0.0532579  0.07132199 0.80215952]
Iteration: 1. Training loss: 2.722E+04.
Series2 Theta :  [1.14761135 0.71547688 0.02420316 0.26453214]
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 195087.06769679225
            Iterations: 1
            Function evaluations: 3
            Gradient evaluations: 1
Iteration: 1. Training loss: 1.951E+05.
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 195087.06769679225
            Iterations: 1
            Function evaluations: 3
            Gradient evaluations: 1
Iteration: 2. Training loss: 1.951E+05.


array([0.5, 0.5])

In [24]:
stablecoins = ["USDC","USDT","DAI","EOSDT","PAXG","XAUT","FRAX","AMPL", "BUSD", "GUSD"]
results = []

for sc in stablecoins:
    results.append(run_dcc_model(datadf["USTC"],datadf[sc]))


Iteration: 1. Training loss: 1.818E+04.
Series1 Theta :  [0.         0.0532579  0.07132199 0.80215952]
Iteration: 1. Training loss: -4.915E+03.
Series2 Theta :  [ 9.69352280e-27  8.24206000e-02 -1.50131815e-02  9.23871249e-01]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 21134.2174377458
            Iterations: 5
            Function evaluations: 3
            Gradient evaluations: 1
Iteration: 1. Training loss: 2.113E+04.
Optimization terminated successfully    (Exit mode 0)
            Current function value: 21134.2174377458
            Iterations: 5
            Function evaluations: 3
            Gradient evaluations: 1
Iteration: 2. Training loss: 2.113E+04.
Iteration: 1. Training loss: 1.818E+04.
Series1 Theta :  [0.         0.0532579  0.07132199 0.80215952]
Iteration: 1. Training loss: -2.807E+03.
Series2 Theta :  [1.00000000e-04 4.63765231e-01 3.41368353e-01 3.65550593e-01]
Optimization terminated successfully    (Exit mode 0)
      

In [25]:
r = []
for i in range(len(results)):
    r.append(["{:.7f}". format(results[i][0]),"{:.7f}". format(results[i][1])])

print("\n####################\n")
for i in range(len(stablecoins)):
    print(stablecoins[i], r[i])


####################

USDC ['0.5000000', '0.5000000']
USDT ['-0.0000000', '0.9364152']
DAI ['0.0000000', '0.8291360']
EOSDT ['0.5000000', '0.5000000']
PAXG ['0.5000000', '0.5000000']
XAUT ['0.5000000', '0.5000000']
FRAX ['0.5000000', '0.5000000']
AMPL ['0.5000000', '0.5000000']
BUSD ['0.5000000', '0.5000000']
GUSD ['0.5000000', '0.5000000']


In [31]:
datanl = np.log(datadf)
datanl.corr()["USTC"]

USTC     1.000000
FRAX     0.241054
AMPL    -0.044120
DAI      0.023874
EOSDT    0.423841
USDC    -0.256138
USDT     0.549692
PAXG     0.588714
XAUT     0.582261
Name: USTC, dtype: float64

In [32]:
datanl = np.log(datadf)
datanl.cov()["USTC"]

USTC     3.151722
FRAX     0.001032
AMPL    -0.013654
DAI      0.000059
EOSDT    0.094137
USDC    -0.000154
USDT     0.000606
PAXG     0.048985
XAUT     0.047090
Name: USTC, dtype: float64